# Autograd: automatic differentiation

The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

In [46]:
import torch

Create a variable:



In [47]:
x = torch.tensor([[1, 2], [3, 4]], requires_grad=True, dtype=torch.float32)
print(x)

tensor([[ 1.,  2.],
        [ 3.,  4.]])


Do an operation of variable:



In [48]:
y = x - 2
print(y)

tensor([[-1.,  0.],
        [ 1.,  2.]])


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [49]:
print(y.grad_fn)

In [50]:
print(x.grad_fn)

None


In [51]:
y.grad_fn

In [52]:
y.grad_fn.next_functions[0][0]

In [53]:
y.grad_fn.next_functions[0][0].variable

tensor([[ 1.,  2.],
        [ 3.,  4.]])

Do more operations on y



In [54]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[  3.,   0.],
        [  3.,  12.]]) tensor(4.5000)


Gradients
---------
let's backprop now
``out.backward()`` is equivalent to doing ``out.backward(torch.Tensor([1.0]))``



In [55]:
out.backward()

print gradients d(out)/dx




In [56]:
print(x.grad)

tensor([[-1.5000,  0.0000],
        [ 1.5000,  3.0000]])


You can do many crazy things with autograd!
> With Great *Flexibility* Comes Great Responsibility

In [57]:
# Dynamic graphs!
x = torch.randn(3)
x = torch.tensor(x, requires_grad=True, dtype=torch.float32)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([  479.6754, -1516.8326,  -287.3617])


In [58]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([  102.4000,  1024.0000,     0.1024])


## Inference

In [59]:
d = 3
t = torch.arange(1, d + 1)

In [60]:
t_v = torch.tensor(t, requires_grad=True, dtype=torch.float32)
theta = torch.ones(d, requires_grad=True)
s = theta @ t_v
s.backward()
print(t_v.grad, theta.grad)

tensor([ 1.,  1.,  1.]) tensor([ 1.,  2.,  3.])


In [61]:
t_v = torch.tensor(t, dtype=torch.float32)
theta = torch.ones(d, requires_grad=True)
s = theta @ t_v
s.backward()
print(t_v.grad, theta.grad, sep='\n')

None
tensor([ 1.,  2.,  3.])


## More stuff

Documentation of ``Variable`` and ``Function`` is at
http://pytorch.org/docs/autograd

